首先，我们对只包含一个变量的数据集建立数值汇总的度量方法。当数据集包含的变量不止一个时，可以对每个变量分别计算其相同的数值测度。

然而，在有两个变量的情况下，我们还将建立变量间相互关系的度量。本章我们将介绍位置、离散程度、形态和相关程度的数值度量。

如果数据来自样本，计算的度量称为**样本统计量**（sample statistics）；
如果数据来自总体，计算的度量称为**总体参数**（population parameters）。

在统计推断中，样本统计量被称为是相应总体参数的点估计量（pointestimator）。在第7章，我们将详细讨论点估计方法。

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, scoreatpercentile

from pathlib import Path
#设置数据文件夹路径
DATA = Path().resolve()  / 'data'

# 位置
## 平均数
说起平均值，或许大家最熟悉的就是：相加再乘以个数，计算公式如下：
$$\displaystyle \bar{x}=\frac{x_1+x_2+\cdots+x_x}{n}  =\frac{1}{n}\displaystyle\sum_{i=1}^n{x_i}$$

- 样本均值：$\bar x$
- 总体均值：$\mu$

这其实只是算术平均值，是众多均值的一种，其实均值有很多种计算方法，或许我们换个角度来理解均值，会比较形象：
> 均值就是从数据的最大值和最小值之间，选取一个值，作为整组数据的代表

虽然计算公式很简单，但实际上均值是统计学体系的基石，因为均值包含的信息量是所有统计量里面最多的，每一个数据都会对均值产生影响。

虽然均值非常重要，但实际上均值能够提供的信息实际上非常少，因此均值一般作为计算其他统计指标的数据。

就和选举一样，选择均值的方式也有很多种，区别在于一个点“你想通过全部数据知道什么”：
- 算术平均值：在合计的意义上保持本质，相加再除以个数：$\displaystyle \frac{(x+y)}{2}$
- 几何平均值：在乘法的意义上保持本质，相乘再开方：$\displaystyle \sqrt{x\times y}$
- 均方根值：先平方，再相加，最后除以个数：$\displaystyle \sqrt{\frac{(x^2 + y^2)}{2}}$
- 调和平均数：处理和“速度”相关的数据，个数除以倒数之和：$\displaystyle \frac{2}{\frac{1}{x}+\frac{1}{y}}$

先不考虑均方根值，下面我们结合一个例子，加深对调和平均数的理解：如果我现在要开车从上海去北京再回来，去程的时速为 $10$，回程的时速为 $90$，平均的时速度是多少？使用调和平均数就可以轻松地算出平均时速为 $\displaystyle \frac{2}{\frac{1}{10}+\frac{1}{90}} = 18$。

为什么我们不能直接使用其他均值计算速度的均值呢？因为“速度”的定义（单位时间内行驶的里程）本身就有一个计算，调和平均数只是还原了这个计算步骤。

假设现在 10 分是大雄的考试分数，90 分是出木杉的考试分数，全班只有两名同学参加补考，那这次考试的四种“平均数”是？

- 算术平均值：$\displaystyle \frac{(10+90)}{2} = 50$
- 几何平均值：$\displaystyle \sqrt{10\times 90} = 30$
- 均方根值：$\displaystyle \sqrt{\frac{(10^2 + 90^2)}{2}} = 64.03$
- 调和平均数：$\displaystyle \frac{2}{\frac{1}{10}+\frac{1}{90}} = 18$

你要是大雄，回家和妈妈汇报成绩的时候，你会用哪个平均数呢？（所以说，统计数字也是会撒谎的）

In [2]:
startSalary = pd.read_csv(DATA / '2012StartSalary.csv')
startSalary['Monthly Starting Salary ($)'].mean()

3940.0

### 截尾均值

在一些场景中，为了避免极值对数据的影响，我们会计算**截尾均值**。

截尾均值最常见的应用场景，就是计算跳水运动选手的分数了，一般我们会去掉一个最高分，去掉一个最低分：

In [3]:
#marathon = pd.read_csv(DATA / 'Marathon.csv')
marathon = pd.DataFrame({'Age': [49,44,50,46,31,27,52,72,33,46,52,24,43,44,43,26,40,57,43,30,50,35,66,59,37,55,64,37,36,31,31,21,56,32,40,43,61,43,50,47]})
marathon.index += 1
marathon.head(5)

,Age
1,49
2,44
3,50
4,46
5,31


In [4]:
trim_mean(marathon['Age'],1/marathon['Age'].count())

43.5

### 加权平均数

在某些场景，我们会使用**加权均值**，结合两个维度：数值本身和权重两个维度。

最常见的应用场景，大概就是我们计算加权平均分的时候，需要结合单科考试分数和该科学分。

在使用加权平均数的时候，需要注意一个假设：每一个平均数的权重值，是人为设定的，如果设定不当，会对数据的计算造成很大的影响。

$$ \bar x = \frac{\sum w_ix_i}{\sum w_i}$$

- $w_i$：第 $i$ 个观测值的权重

In [5]:
np.average(marathon['Age'], weights=marathon.index)

43.645121951219515

### 几何平均数

$$\bar{x}_g = \sqrt[n]{x_1 x_2 \cdots x_n} = (x_1 x_2 \cdots x_n)^{\frac{1}{n}}$$

在财务、投资和银行业的问题中， 几何平均数的应用尤为常见。当你想确定过去几个连续时期的平均变化率时 都能应用几何平均数。

几何平均数可以用于发生在所有时间长度的连续时期的任何数量的变化率。

特点在于：**在乘法的意义上保持本质**

## 中位数

将数据从小到大排序
- 对奇数个观测数：中位数是中间的数值
- 对偶数个观测数：中位数是中间两个数值的平均数

一组数据中间位置上的代表值. 其特点是不受数据极端值的影响. 对于具有偏态分布的数据, 中位数的代表性要比均值好。

In [6]:
marathon['Age'].median()

43.0

## 众数

一组数据分布出现次数最多的数据，一组数据可能有一个众数，也可能有两个或多个众数，也可能没有众数。

In [7]:
marathon['Age'].mode()

0    43
Name: Age, dtype: int64

## 百分位数

提供数据如何散布在从最小值到最大值上的区间上的信息

$$L_p = \frac{p}{100}(n+1)$$

- 若 $i$ 不是整数 则向上取整数 大于 $i$ 的下一个整数表示第 $p$ 百分位数的位置
- 若 $i$ 是整数 则第 $p$ 百分位数是第 $i$ 项和第 $i+1$ 项数据的平均值

## 四分位数


把所有数值由小到大排列，按照个数平分成四等份，处于三个分割点位置的数值就是四分位数。

- $Q_1$ = 第一四分位数，或第 25 百分位数
- $Q_2$ = 第二四分位数，或第 50 百分位数（也是中位数）
- $Q_3$ = 第三四分位数，或第 75 百分位数

1. 当数据集中含有极端值时 使用中位数作为中心位置的度量比平均数更合适
2. 其他常用百分位数是五分位数和十分位数

In [3]:
fish_data_3 = np.array([1,2,3,4,5,6,7,8,9])
scoreatpercentile(fish_data_3, 25)

3.0

In [5]:
scoreatpercentile(fish_data_3, 75)

7.0

## 变异程度的度量（离散程度的度量）

### 极差

极差 = 最大值 - 最小值

四分位数间距

第三四分位数 $Q_3$ 与第一四分位数 $Q_1$ 的差距又称四分位距 $IQR$。

$$IQR = Q_3 - Q_1$$


### 异常值的检测

1. 标准化数值来确定异常值
2. 以第一四分位数和第三四分位数以及四分位数间距

异常值的判定和四分位距有关：如果一个值小于第一个四分位数减去 1.5 倍的 IQR 或者大于 Q3 加上 1.5 乘以 IQR，则这个数就被认为是异常值。

- $< Q1 - 1.5 * IQR$
- $> Q3 + 1.5 * IQR$


### 五数概括法

- 最小值
- 第一四分位数
- 中位数
- 第三四分位数
- 最大值

## 方差：度量数据的变异程度

前面的所有计算结果（各种均值、中位数、众数、四分位数等）在统计学上来说，其实都是“统计量”。这些统计量，其实都是在说明一件事：数据与均值的关系。

我记得之前听过一首打油诗：

> 张村有个张千万，隔壁九个穷光蛋，平均起来算一算，人人都是张百万。

这首打油诗生动形象地展示了算术平均数的局限性。基于平均数，我们还可以尝试计算这些统计量：

- 离差（Deviation）样本值减去均值。$D_i = x_i-\bar{x}$
- 平均离差：$\displaystyle \sum{\frac{D_i}{n}}=\frac{1}{n}\sum_{i=1}^n{x_i-\bar{x}}$
- 绝对离差：$|D_i|=|x_i-\bar{x}|$
- 平方离差（离差平方和/平方和）：$SS = {D_i}^2=(x_i-\bar{x})^2$

接下来我们要计算的方差，就是平方离差除以样本数量。

总体方差
$$\sigma^2 = \frac {\sum(x_i - \mu)^2}{N}$$

样本方差
$$s^2 = \frac {\sum(x_i - \bar x)^2}{n-1}$$

In [3]:
#bus = pd.read_csv(DATA / 'Bus.csv')
bus = pd.DataFrame({'Bus A': [40, 40, 50, 60, 60], 'Bus B': [1, 2, 60, 88, 99]})
bus.index += 1
bus

,Bus A,Bus B
1,40,1
2,40,2
3,50,60
4,60,88
5,60,99


计算出两组的方差之后，我们会发现，B 组的方差明显大于 A 组的方差，那方差实际上意味着什么呢？

不知道你们有没有冲浪，实际上方差和冲浪有相似之处。海水水位其实不是冲浪选手需要关心的问题，海浪高度变化，才是冲浪选手最关心的。

实际上，我们可以将均值理解成海水水位，而方差理解成海浪高度，方差越大，潮起潮落之间的海浪高度差越大。

在上面公交车等候时间的例子中，我们当然希望方差越小越好，因为我们希望每趟车的间隔差不多长，才能确保我们的等候时间是在合理范围内的。

除了方差之外，标准差其实是统计学中更常用的概念。

$$\displaystyle SD = \sqrt{Var} = \sqrt{\frac{\sum(x_i-\bar{x})^2}{n}}$$

In [4]:
bus['Bus A'].var(ddof=0)

80.0

In [5]:
bus['Bus B'].var(ddof=0)

1730.0

那为什么我们有了方差之后，还需要标准差呢？很简单，我们可以用 B 组数据来做演示，分别计算 B 组数据分别加上 5 和乘以 5 的方差和标准差。

In [7]:
print((bus['Bus B']+5).var(ddof=0))
print((bus['Bus B']+5).std(ddof=0))
print((bus['Bus B']*5).var(ddof=0))
print((bus['Bus B']*5).std(ddof=0))

1730.0
41.593268686170845
43250.0
207.9663434308542


细心的朋友会发现：
- 加上 5 的方差和标准差没有变化
- 乘以 5 的方差是原来方差的 $5^2 = 25$ 倍，标准差是原来的 5 倍

由于方差计算过程中需要去平方，因此会放大数据原本的差异，在计算结果之后，我们需要开方，抹除被放大的差异。

### 贝塞尔校正

通常，抽样中会低估了总体中差异性的数量，因为抽样往往是总体居于中间的值，特别是在正态分布中多数值居于中间位置，

因此我们从正态分布的总体中抽样时，多数值也在此处附近，因为多数值在这个区域内，

因此抽样中的差异性将少于整个总体的差异性，

为了纠正这一现象，我们使用贝塞尔校正系数，我们把除以 n 用除以 n 减 1 代替，在方差的处理中也是一样的

$$
\begin{aligned}
总体方差：σ^2&=\displaystyle\frac{\sum(X-\mu)^2}{N} \\
样本方差：s^2&=\displaystyle\frac{\sum(X-\bar{X})^2}{n-1} \\
\end{aligned}
$$

其中:
- $X$：样本值
- $\mu$：总体均值, 总体期望值
- $\bar{X}$：样本均值, 样本期望值
- $N$：样本总数
- $n$：样本个数

In [8]:
MEAN = startSalary['Monthly Starting Salary ($)'].mean()
startSalary['均值'] = MEAN
startSalary['离差'] = startSalary['Monthly Starting Salary ($)'] - startSalary['均值']
startSalary['离差平方'] = startSalary['离差']**2
startSalary

,Graduate,Monthly Starting Salary ($),均值,离差,离差平方
0,1,3850,3940.0,-90.0,8100.0
1,2,3950,3940.0,10.0,100.0
2,3,4050,3940.0,110.0,12100.0
3,4,3880,3940.0,-60.0,3600.0
4,5,3755,3940.0,-185.0,34225.0
5,6,3710,3940.0,-230.0,52900.0
6,7,3890,3940.0,-50.0,2500.0
7,8,4130,3940.0,190.0,36100.0
8,9,3940,3940.0,0.0,0.0
9,10,4325,3940.0,385.0,148225.0


In [9]:
#方差
VAR = startSalary['离差平方'].sum()/(startSalary['Graduate'].count() - 1)
VAR

27440.909090909092

In [12]:
#startSalary['Monthly Starting Salary ($)'].var()
# 不使用贝塞尔校正，传参 ddof=0
startSalary['Monthly Starting Salary ($)'].var(ddof=0)

25154.166666666668

## 标准差

标准差是方差的正平方根，与数据的单位相同

由于方差计算过程中需要去平方，因此会放大数据原本的差异，在计算结果之后，我们需要开方，抹除被放大的差异。

总体标准差
$$\sigma = \sqrt{\sigma^2} = \sqrt\frac{\sum(X-\mu)^2}{N}$$

样本标准差

$$s = \sqrt{s^2} = \sqrt\frac{\sum(X-\bar{X})^2}{n-1}$$

In [13]:
#标准差
SD = VAR**0.5
SD

165.65297791138283

In [14]:
startSalary['Monthly Starting Salary ($)'].std()

165.65297791138283

## 标准差系数

$$(\frac{标准差}{平均数} \times 100)%$$

说明 样本标准差仅为样本平均数的 ？%

比较均值和标准差都不同的变量的变异程度时，可以使用标准差系数

In [15]:
CV = SD/MEAN
CV

0.042043903023193614

# 分布形态、相对位置的度量以及异常值的检测

## 分布形态：左偏- 右偏+ 对称0

偏度计算： 偏度 = $ \frac {n}{(n-1)(n-2)} \sum (\frac {x_i - \bar x}{s})^3 $

## z-分数 数据集中的数值的相对位置

### 偏差值

中国的考试成绩计算方式，是算绝对数，而日本的考试成绩计算方式，计算的是“偏差值”，计算的过程中，就会用到标准差：

$$ 偏差值 = 50 + 10 \times \frac{个人成绩-平均成绩}{标准差}$$

如果你的个人成绩是 60 分，平均成绩是 50 分，标准差是 5 分，则偏差值为：

$$ 偏差值 = 50 + 10 \times \frac{60-50}{5} = 70$$

如果你的个人成绩是 70 分，平均成绩是 60 分，标准差是 5 分，则偏差值为：

$$ 偏差值 = 50 + 10 \times \frac{70-60}{5} = 70$$

实际上，日本考试成绩计算的过程，就是“标准化”的过程。


首先，我们得先理解，什么叫**标准化**？统计学是这样定义**标准化**的：
> 标准化就是将分布中的各个原始取值转换为具有标准差单位的 $z$ 分数的过程。

下一节我们会提到 $z$ 分数，现在我们先不用纠结 $z$ 分数的定义，我们先理解**标准化**这个概念。

下面我们用一个简单的例子来讲解**标准化**这个过程：

### 标准化

大雄有两门考试
- 英语的成绩是 75 分，班级平均分是 70 分，标准差是 5
- 体育的成绩是 10 个，班级平均个数是 9 个，标准差是 0.5

问：哪门成绩考得比较好？

由于两门考试的测度标准不一致，实际上是无法比较的（而且体育和英语也不应该互相比较）

但统计学家发明了**标准化**这个概念，来对测度标准不一致的数据，根据其**位于分布的位置**进行对比。

标准化的过程如下：

$$\displaystyle z_i = \frac{原始取值 - 均值}{标准差} = \frac {x_i -\bar x}{s}$$

- $\bar x$ 为样本均值
- $s$ 为样本标准差

英语成绩标准化的结果是：

$$\displaystyle \frac{75 - 70}{5} = 1$$

体育成绩标准化的结果是：

$$\displaystyle \frac{10 - 9}{0.5} = 2$$

我们无法对 75 分和 10 个直接进行比较，但是我们却可以比较标准化之后的结果。体育的标准化结果比英语大，所以我们可以判定：体育这门课考得比较好。

### $z$ 分数

- $z$ 分数是以标准差为单位度量的分布中，一个给定取值与均值之间的距离数。
- $z$ 分数有助于理解个别取值与分布中其他取值之间的位置关系。

实际上，标准化的过程就是 $z$ 分数的计算公式

$$
\begin{aligned}
\displaystyle
z &= \frac{原始取值 - 均值}{标准差} \\
  &= \frac{X - \mu}{\sigma} \\
  &= \frac{X - \bar{X}}{s}
\end{aligned}
$$


其中
- $X$：原始取值
- $\mu$：总体均值
- $\sigma$：总体标准差
- $\bar{X}$：样本均值
- $s$：样本标准差

- 如果原始值大于均值，则 $z$ 分数为正
- 如果原始值小于均值，则 $z$ 分数为负

## 解释 $z$ 分数

- $z$ 分数可以看出个别取值**相对**于其他取值的大小，但没有包含绝对值信息
- $z$ 分数常用于确定**正态分布**数据集合的百分位数值，如果数据不服从正态分布，则无法通过 $z$ 分数来确定百分位数

我们再回到上面考试成绩的例子，由于我们计算得出英语的 $z$ 分数是 1，而体育的 $z$ 分数是 2。**假设**两门考试成绩的分布都**服从正态分布**，通过查阅 [$z$ 表格](https://www.ztable.net/)可得：
- 英语这门课，我们比 84.13% 的人考的都好
- 体育这门课，我们比 97.72% 的人考的都好

我们可以用`statistics`库来计算 $z$ 分数

In [16]:
startSalary['z 分数'] = (startSalary['Monthly Starting Salary ($)'] - startSalary['均值'])/SD
startSalary

,Graduate,Monthly Starting Salary ($),均值,离差,离差平方,z 分数
0,1,3850,3940.0,-90.0,8100.0,-0.543304
1,2,3950,3940.0,10.0,100.0,0.060367
2,3,4050,3940.0,110.0,12100.0,0.664039
3,4,3880,3940.0,-60.0,3600.0,-0.362203
4,5,3755,3940.0,-185.0,34225.0,-1.116792
5,6,3710,3940.0,-230.0,52900.0,-1.388445
6,7,3890,3940.0,-50.0,2500.0,-0.301836
7,8,4130,3940.0,190.0,36100.0,1.146976
8,9,3940,3940.0,0.0,0.0,0.000000
9,10,4325,3940.0,385.0,148225.0,2.324136


## 切比雪夫定理

与平均数的距离在 $z$ 个标准差之内的数据项所占比例至少为

$$( 1 - \frac{1}{z^2})$$

- $z$ 是大于 1 的任意实数

切比雪夫定理的优点之一就是，它**适用于任何数据集**而不论其数据分布的形状。

当数据近似于钟形分布时，就可以运用经验法则（empirical rule）来确定与平均数的距离在某个特定个数的标准差之内的数据值所占的比例。对于具有钟形分布的数据
- 大约 68% 的数据值与平均数的距离在 1 个标准差之内
- 大约 95% 的数据值与平均数的距离在 2 个标准差之内
- 几乎所有的数据与平均数的距离在 3 个标准差之内

# 两变量间关系的度量

## 协方差

[如何通俗地解释协方差](https://www.bilibili.com/video/BV1gY4y187TL)

总体协方差

$$s_{xy} = \frac {\sum(x_i - \bar x)(y_i - \bar x)}{n-1}$$

In [17]:
stereo = pd.read_csv(DATA / 'Stereo.csv')
stereo['x_i - x_bar'] = stereo['No. of Commercials'] - stereo['No. of Commercials'].mean()
stereo['y_i - y_bar'] = stereo['Sales Volume'] - stereo['Sales Volume'].mean()
stereo['(x_i - x_bar)(y_i - y_bar)'] = stereo['x_i - x_bar'] * stereo['y_i - y_bar']
stereo

,Week,No. of Commercials,Sales Volume,x_i - x_bar,y_i - y_bar,(x_i - x_bar)(y_i - y_bar)
0,1,2,50,-1.0,-1.0,1.0
1,2,5,57,2.0,6.0,12.0
2,3,1,41,-2.0,-10.0,20.0
3,4,3,54,0.0,3.0,0.0
4,5,4,54,1.0,3.0,3.0
5,6,1,38,-2.0,-13.0,26.0
6,7,5,63,2.0,12.0,24.0
7,8,3,48,0.0,-3.0,-0.0
8,9,4,59,1.0,8.0,8.0
9,10,2,46,-1.0,-5.0,5.0


In [18]:
s_xy = stereo['(x_i - x_bar)(y_i - y_bar)'].sum() / (stereo['Week'].count() - 1)
s_xy

11.0

In [19]:
df = pd.read_csv(DATA / 'Stereo.csv')
del df['Week']

In [20]:
np.cov(df.T)

array([[ 2.22222222, 11.        ],
       [11.        , 62.88888889]])

In [21]:
np.var(df, axis=0, ddof=1)

No. of Commercials     2.222222
Sales Volume          62.888889
dtype: float64

### 协方差解释

$ s_{xy} $为正值表示x和y之间存在正的线性关系：随着x的值的增加，y的值也增加 如果 $ s_{xy} $为负值 则表示x和y为负的线性相关关系：随着x的值的增加 y的值减少

### 局限性
协方差值受数值单位影响。假设研究人体重与身高关系，身高用厘米测出数值比英寸大得多，为了避免这种情况，可以使用相关系数。

## 相关系数

## 皮尔逊积矩相关系数：样本数据 
$ r_{xy} = \frac {s_{xy}} {s_xs_y}$
- $r_{xy}$ 表示样本相关系数
- $s_{xy}$ 表示样本协方差
- $s_y s_x$ 分别表示x的样本标准差 y的样本标准差

In [22]:
stereo

,Week,No. of Commercials,Sales Volume,x_i - x_bar,y_i - y_bar,(x_i - x_bar)(y_i - y_bar)
0,1,2,50,-1.0,-1.0,1.0
1,2,5,57,2.0,6.0,12.0
2,3,1,41,-2.0,-10.0,20.0
3,4,3,54,0.0,3.0,0.0
4,5,4,54,1.0,3.0,3.0
5,6,1,38,-2.0,-13.0,26.0
6,7,5,63,2.0,12.0,24.0
7,8,3,48,0.0,-3.0,-0.0
8,9,4,59,1.0,8.0,8.0
9,10,2,46,-1.0,-5.0,5.0


In [23]:
#s_x
s_x = pow(pow(stereo['x_i - x_bar'],2).sum() / (stereo['Week'].count() - 1),0.5)
#s_y
s_y = pow(pow(stereo['y_i - y_bar'],2).sum() / (stereo['Week'].count() - 1),0.5)
#r_{xy}
r_xy = s_xy / (s_x * s_y)
r_xy

0.9304905807411791

In [24]:
np.corrcoef(df.T)

array([[1.        , 0.93049058],
       [0.93049058, 1.        ]])

## 总体数据
$ \rho_{xy} = \frac {\sigma_{xy}} {\sigma_x\sigma_y}$
- $\rho_{xy}$ 表示总体相关系数
- $\sigma_{xy}$ 表示总体协方差
- $\sigma_y$ 和 $\sigma_x$ 分别表示 $x$ 的总体标准差，$y$ 的总体标准差

- 样本相关系数是总体相关系数的点估计量
- [-1, 1] 且与0之间的距离越来越靠近时，表明线性关系越来越弱，接近于 -1 和 1 时，表示强的线性关系

### [协方差分析](https://www.statology.org/ancova-python/)
在进行协方差分析之前，我们需要先对变量进行区分：

- 自变量
    - 解释变量（独立变量/预测变量）：解释因变量变化原因，我们主要研究的的变量
    - 协调变量（协变量）：与解释变量共同对因变量产生影响，但不是我们主要研究的变量
- 因变量（响应变量/结果变量）：受到自变量影响的变量。

假设我们现在有一个数据集，有一个定类变量 `technique` 和两个定量变量：`current_grade` 和 `exam_score`。

In [25]:
df = pd.DataFrame({'technique': np.repeat(['A', 'B', 'C'], 5),
                   'current_grade': [67, 88, 75, 77, 85, 92, 69, 77, 74, 88, 96, 91, 88, 82, 80],
                   'exam_score': [77, 89, 72, 74, 69, 78, 88, 93, 94, 90, 85, 81, 83, 88, 79]})
#view data 
df.head()

,technique,current_grade,exam_score
0,A,67,77
1,A,88,89
2,A,75,72
3,A,77,74
4,A,85,69


其中：
- 自变量：定类变量 `technique`、定量变量：`current_grade`
- 因变量：定量变量 `exam_score`

假设我们没有了解过协方差分析，其实也可以对这个数据集进行分析，我们只需要对定量变量：`current_grade` 进行装箱处理，就可以把两个自变量都转换成定类变量，这样我们就能愉快滴进行双因子方差分析了。

其实协方差可以算是多因素方差分析（后面会学到）的一个特例，此时，如果我们不对定量变量：`current_grade` 进行装箱处理，使用协方差分析，也可以完成分析任务。

In [26]:
#perform ANCOVA
from pingouin import ancova

ancova(data=df, dv='exam_score', covar='current_grade', between='technique')

,Source,SS,DF,F,p-unc,np2
0,technique,390.575130,2,4.809973,0.031556,0.466536
1,current_grade,4.193886,1,0.103296,0.753934,0.009303
2,Residual,446.606114,11,NaN,NaN,NaN


From the ANCOVA table we see that the p-value (p-unc = “uncorrected p-value”) for study technique is 0.03155.

Since this value is less than 0.05, we can reject the null hypothesis that each of the studying techniques leads to the same average exam score, even after accounting for the student’s current grade in the class.